In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df=pd.read_csv('countries of the world.csv')

In [ ]:
df.columns = (
    df.columns
    .str.strip()
    .str.lower()
    .str.replace(r'[^\w\s]', '', regex=True)       # remove special characters
    .str.replace(r'\s+', '_', regex=True)          # spaces to underscores
    .str.replace(r'_+', '_', regex=True)           # multiple underscores to single
    .str.rstrip('_')                               # remove trailing underscore
)

# Convert numeric columns from string to float (handling commas as decimal points)
columns_to_convert = df.columns.drop(['country', 'region'])  # exclude non-numeric columns

for col in columns_to_convert:
    df[col] = df[col].astype(str).str.replace(',', '.', regex=False)
    df[col] = pd.to_numeric(df[col], errors='coerce')

In [ ]:
print(df.columns)

Index(['country', 'region', 'population', 'area_sq_mi',
       'pop_density_per_sq_mi', 'coastline_coastarea_ratio', 'net_migration',
       'infant_mortality_per_1000_births', 'gdp_per_capita', 'literacy',
       'phones_per_1000', 'arable', 'crops', 'other', 'climate', 'birthrate',
       'deathrate', 'agriculture', 'industry', 'service'],
      dtype='object')


In [ ]:
df["region"].unique()

array(['ASIA (EX. NEAR EAST)         ',
       'EASTERN EUROPE                     ',
       'NORTHERN AFRICA                    ',
       'OCEANIA                            ',
       'WESTERN EUROPE                     ',
       'SUB-SAHARAN AFRICA                 ', 'LATIN AMER. & CARIB    ',
       'C.W. OF IND. STATES ', 'NEAR EAST                          ',
       'NORTHERN AMERICA                   ',
       'BALTICS                            '], dtype=object)

In [ ]:
# Strip leading/trailing spaces and apply consistent formatting
df['region'] = df['region'].str.strip().str.title()


In [ ]:
# Replace abbreviated region names with more descriptive ones
df['region'] = df['region'].replace({
    'Latin Amer. & Carib': 'Latin America & Caribbean',
    'C.W. Of Ind. States': 'Commonwealth Of Independent States'
})

In [ ]:
# Check the cleaned-up region column
print(df['region'].unique())

['Asia (Ex. Near East)' 'Eastern Europe' 'Northern Africa' 'Oceania'
 'Western Europe' 'Sub-Saharan Africa' 'Latin America & Caribbean'
 'Commonwealth Of Independent States' 'Near East' 'Northern America'
 'Baltics']


In [ ]:
# To display the values without scientific notation in the notebook
pd.set_option('display.float_format', '{:.0f}'.format)

# Check the DataFrame
print(df['population'].head(20))


0      31056997
1       3581655
2      32930091
3         57794
4         71201
5      12127071
6         13477
7         69108
8      39921833
9       2976372
10        71891
11     20264082
12      8192880
13      7961619
14       303770
15       698585
16    147365352
17       279912
18     10293011
19     10379067
Name: population, dtype: int64


In [ ]:
nulls_before_imputation = df.isnull().sum()
nulls_before_imputation = nulls_before_imputation[nulls_before_imputation > 0]
nulls_before_imputation

,0
net_migration,3
infant_mortality_per_1000_births,3
gdp_per_capita,1
literacy,18
phones_per_1000,4
arable,2
crops,2
other,2
climate,22
birthrate,3


In [ ]:
# Compute skewness for all numeric columns
skewness = df.select_dtypes(include=['float64', 'int64']).skew().sort_values(ascending=False)

# Suggest imputation method based on skewness
imputation_strategy = skewness.apply(lambda x: 'Median' if abs(x) > 1 else 'Mean')

# Combine into a DataFrame for display
imputation_recommendation = pd.DataFrame({
    'Skewness': skewness,
    'Recommended Imputation': imputation_strategy
})

In [ ]:
imputation_recommendation

,Skewness,Recommended Imputation
population,9,Median
pop_density_per_sq_mi,8,Median
coastline_coastarea_ratio,8,Median
area_sq_mi,6,Median
crops,3,Median
deathrate,2,Median
infant_mortality_per_1000_births,1,Median
agriculture,1,Median
gdp_per_capita,1,Median
arable,1,Median


In [ ]:
# Subset to the columns that originally had missing values
columns_with_nulls = nulls_before_imputation.index

# Separate columns by recommended imputation method
median_impute_cols = [col for col in columns_with_nulls if imputation_recommendation.loc[col, 'Recommended Imputation'] == 'Median']
mean_impute_cols = [col for col in columns_with_nulls if imputation_recommendation.loc[col, 'Recommended Imputation'] == 'Mean']

# Apply imputation only to those columns and round to 2 decimals
df[median_impute_cols] = df[median_impute_cols].fillna(df[median_impute_cols].median().round(2))
df[mean_impute_cols] = df[mean_impute_cols].fillna(df[mean_impute_cols].mean().round(2))

# Verify again if any nulls remain in those specific columns
null_check_subset = df[columns_with_nulls].isnull().sum()
null_check_subset = null_check_subset[null_check_subset > 0]



In [ ]:
null_check_subset

,0


In [ ]:
df["climate"].unique()

array([1.  , 3.  , 2.  , 2.14, 4.  , 1.5 , 2.5 ])

In [ ]:
# Round the 'climate' column values up to the next integer
df['climate'] = df['climate'].apply(np.ceil).astype(int)

# Check the result
print(df['climate'].head(10))

0    1
1    3
2    1
3    2
4    3
5    3
6    2
7    2
8    3
9    4
Name: climate, dtype: int64


In [ ]:
df.rename(columns={'other': 'non_agricultural_land'}, inplace=True)


In [ ]:
df['calc_non_agricultural'] = 100 - df['arable'] - df['crops']
diff = (df['non_agricultural_land'] - df['calc_non_agricultural']).abs()
print("Max deviation:", diff.max())


Max deviation: 6.909999999999997


In [ ]:
df.drop('calc_non_agricultural',axis=1,inplace=True)

In [ ]:
df.head()

,country,region,population,area_sq_mi,pop_density_per_sq_mi,coastline_coastarea_ratio,net_migration,infant_mortality_per_1000_births,gdp_per_capita,literacy,phones_per_1000,arable,crops,non_agricultural_land,climate,birthrate,deathrate,agriculture,industry,service
0,Afghanistan,Asia (Ex. Near East),31056997,647500,48,0,23,163,700,36,3,12,0,88,1,47,20,0,0,0
1,Albania,Eastern Europe,3581655,28748,125,1,-5,22,4500,86,71,21,4,74,3,15,5,0,0,1
2,Algeria,Northern Africa,32930091,2381740,14,0,-0,31,6000,70,78,3,0,97,1,17,5,0,1,0
3,American Samoa,Oceania,57794,199,290,58,-21,9,8000,97,260,10,15,75,2,22,3,0,0,1
4,Andorra,Western Europe,71201,468,152,0,7,4,19000,100,497,2,0,98,3,9,6,0,0,1


In [ ]:
# If the first column is unnamed (often saved index)
if df.columns[0] == '' or df.columns[0].lower().startswith('unnamed'):
    df.drop(df.columns[0], axis=1, inplace=True)
# Ensure index-based columns are not retained
df.reset_index(drop=True, inplace=True)

In [ ]:
df.to_csv('countries of the world_cleaned_new.csv')